In [16]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
from datetime import date
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from (selenium).webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup

options = Options()
options.add_experimental_option('detach', True)
options.add_experimental_option('excludeSwitches', ['enable-logging'])

# 크롬드라이버 최신 설치
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 데이터를 담을 빈 리스트들을 생성
corp_names = [] # 회사명
selected_fields = [] # 선정분야
industry_scales = [] # 업종
locations = [] # 위치
employees = [] # 직원수 
interests = [] # 관심기업

# 이건 페이지 전체를 담는 코드
for page in range(1, 2779):  # 총 2페이지에 대해, 이 숫자만 변경하면 됨 -> ex for page in range(1, 2111)
    
    # 페이지 번호가 URL에 포함되어 있다면, 이렇게 페이지 번호를 변경
    url = f'https://www.work.go.kr/jobyoung/smallGiants/corpInfoSrchList.do?subIndTpCd=&employNum=&smlgntCoClcd2=&coGbCd=small&smlgntCoClcd1=&ctrlOrgCd=&resultCnt=10&isEmptyHeader=&kedBusiSize=&_csrf=4be26908-5572-4bd5-9f48-d9f6efafe541&searchYear=2019&searchOn=Y&indTpCd=&isChkLocCall=&pageIndex={page}&sortField=busiNm&keyword=&region=&sortOrderBy=ASC&superIndTpCd='
    driver.get(url)
    
    for i in range(1, 11):  # 각 페이지에 10행이 존재해서 for문에서 11까지
        try:
            corp_name = driver.find_element(By.XPATH, f'//*[@id="frm"]/div[2]/table/tbody/tr[{i}]/td[1]').text
            corp_names.append(corp_name)

            selected_field = driver.find_element(By.XPATH, f'//*[@id="frm"]/div[2]/table/tbody/tr[{i}]/td[2]').text
            selected_fields.append(selected_field)

            industry_scale = driver.find_element(By.XPATH, f'//*[@id="frm"]/div[2]/table/tbody/tr[{i}]/td[3]/p').text
            industry_scales.append(industry_scale)

            location = driver.find_element(By.XPATH, f'//*[@id="frm"]/div[2]/table/tbody/tr[{i}]/td[3]/div').text
            locations.append(location)

            employee = driver.find_element(By.XPATH, f'//*[@id="frm"]/div[2]/table/tbody/tr[{i}]/td[4]').text
            employees.append(employee)

            interest = driver.find_element(By.XPATH, f'//*[@id="frm"]/div[2]/table/tbody/tr[{i}]/td[5]/p[1]').text
            interests.append(interest)
            
        except Exception as e:
            print(f"Error occurred at page {page} row {i} : ", e)

# 리스트들을 데이터프레임으로 변환
# 컬럼명은 영어로 바꿔도 됨
    df = pd.DataFrame({
        '기업명': corp_names, 
        '선정분야': selected_fields,
        '업종/규모': industry_scales,
        '근로자수': locations,
        '소재지': employees,
        '관심기업': interests })
    df = df.reset_index(drop=True)

df.to_csv('2023_gang.csv', index=False)

In [20]:
df1 = df.copy()

In [21]:
df['근로자수'].unique()

array(['중소기업 0명', '중소기업 22명', '중소기업 60명', '중소기업 36명', '중소기업 27명',
       '중소기업 103명', '중소기업 199명', '중소기업 19명', '중소기업 18명', '중소기업 75명', '0명',
       '중소기업 25명', '중견기업 128명', '중소기업 31명', '중소기업 83명', '중소기업 157명',
       '중견기업 135명', '중소기업 13명', '중소기업 194명', '중소기업 138명', '중소기업 148명',
       '중소기업 20명', '중소기업 133명', '중소기업 32명', '중견기업 122명', '중소기업 68명',
       '중소기업 81명', '중소기업 29명', '중소기업 35명', '중소기업 30명', '중소기업 24명',
       '중소기업 50명', '중소기업 12명', '중견기업 366명', '중소기업 8명', '중소기업 16명', '28명',
       '중소기업 58명', '중소기업 40명', '중소기업 43명', '중소기업 21명', '중소기업 46명',
       '중소기업 14명', '중소기업 99명', '중소기업 117명', '중소기업 26명', '중소기업 86명',
       '중소기업 11명', '중소기업 38명', '중소기업 59명', '중견기업 333명', '중소기업 94명',
       '중소기업 47명', '중소기업 10명', '중소기업 17명', '중소기업 65명', '중견기업 203명',
       '중소기업 114명', '중견기업 258명', '중견기업 125명', '중소기업 71명', '중견기업 393명',
       '중소기업 256명', '중소기업 125명', '중소기업 23명', '중소기업 132명', '중소기업 66명',
       '중소기업 37명', '중소기업 62명', '중소기업 82명', '중소기업 100명', '중소기업 130명',
       '중소기업 162명', '중견기업 48

In [25]:


# 근로자수에서 숫자 부분을 추출 (콤마 포함)
df['직원수'] = df['근로자수'].str.extract('(\d+,*\d*)')

# 콤마 제거
df['직원수'] = df['직원수'].str.replace(',', '')

# '직원수' column을 integer 형으로 변환
df['직원수'] = df['직원수'].astype(int)


In [38]:
def get_company_size(text):
    parts = text.split(' ')
    return parts[0] if len(parts) > 1 else '알수없음'

df['기업규모'] = df['근로자수'].apply(get_company_size)

In [39]:
df['기업규모'].unique()


array(['중소기업', '알수없음', '중견기업', '기타', '대기업'], dtype=object)

In [42]:
# '소재지'에서 띄어쓰기 기준 첫 번째 부분을 '소재지(대)'에 저장
df['소재지(대)'] = df['소재지'].str.split(' ').str[0]

# '소재지'에서 띄어쓰기 기준 두 번째 부분을 '소재지(중)'에 저장
df['소재지(중)'] = df['소재지'].str.split(' ').str[1]

In [44]:
df['소재지(대)'].unique()

array(['서울', '강원', '전남', '충남', '전북', '경기', '대전', '경남', '인천', '충북', '광주',
       '경북', '제주', '대구', '부산', '울산', '세종'], dtype=object)

In [49]:
df['소재지']

array(['서울 강남구', '강원 태백시', '전남 여수시', '충남 서천군', '전북 군산시', '전남 영암군',
       '경기 안산시 상록구', '강원 원주시', '강원 홍천군', '대전 유성구', '경남 산청군', '경기 평택시',
       '서울 구로구', '경기 화성시', '인천 남동구', '서울 서초구', '경기 수원시 영통구', '서울 영등포구',
       '인천 서구', '경기 성남시 분당구', '충북 청주시 청원구', '충남 보령시', '광주 광산구', '경기 군포시',
       '경기 수원시 권선구', '강원 강릉시', '광주 북구', '경북 문경시', '제주 제주시', '서울 성동구',
       '경남 창원시 마산회원구', '충북 충주시', '서울 마포구', '경기 남양주시', '경기 안성시', '경기 이천시',
       '경기 고양시 일산서구', '경북 칠곡군', '서울 강동구', '인천 미추홀구', '대구 달서구', '경남 김해시',
       '부산 사하구', '경기 안산시 단원구', '경기 시흥시', '충북 진천군', '경기 부천시', '울산 남구',
       '서울 양천구', '대구 동구', '전남 광양시', '경기 용인시 기흥구', '부산 강서구', '경북 구미시',
       '충북 청주시 흥덕구', '전남 장성군', '부산 북구', '충남 천안시 서북구', '경남 함안군', '울산 북구',
       '경기 포천시', '부산 영도구', '충남 아산시', '서울 중구', '경기 파주시', '대전 동구', '경기 의왕시',
       '서울 종로구', '대전 서구', '서울 송파구', '부산 부산진구', '경남 창원시 성산구', '부산 사상구',
       '경남 양산시', '강원 횡성군', '경기 김포시', '서울 금천구', '울산 울주군', '경북 경산시',
       '인천 부평구', '대구 북구', '경남 진주시', '대구 달성군', '경북 포항시 북구', '경기 광주시',
    

In [50]:
df['관심기업건수'] = df['관심기업'].str.replace('건', '').astype(int)

In [52]:
#df.to_csv('gang2.csv', index=False)